# 네이버 지식인 크롤링

In [64]:
import requests
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from urllib.parse import quote
import time
import re

In [65]:
driver = webdriver.Chrome('data/chromedriver')

In [66]:
base_url ='https://kin.naver.com/search/list.naver?query='
url = base_url + quote('여친선물')
driver.get(url)
time.sleep(2)

In [67]:
html = driver.page_source
soup = BeautifulSoup(html,'html.parser')

In [68]:
lis = soup.select('.basic1 li')
len(lis)

10

In [69]:
li =lis[0]
sub_url = li.select_one('a._searchListTitleAnchor')['href']
sub_url

'https://kin.naver.com/qna/detail.naver?d1id=8&dirId=80101&docId=395035694&qb=7Jes7Lmc7ISg66y8&enc=utf8&section=kin&rank=1&search_sort=0&spq=0'

In [70]:
driver.get(sub_url)

In [71]:
count = int(driver.find_element_by_css_selector('._answerCount.num').text)
anwer_page = (count-1) // 5 #5개일때 문제생겨서 한개뻄
count, anwer_page

(8, 1)

In [72]:
for i in range(anwer_page):
    driver.find_element_by_id('nextPageButton').click()
    time.sleep(2)

In [73]:
html = driver.page_source
soup = BeautifulSoup(html,'html.parser')

In [74]:
divs = soup.select('.answer-content__item._contentWrap._answer')
len(divs)

8

In [75]:
div = divs[0]
ps = div.select('.se-module.se-module-text p')
text=[]
for p in ps:
    text.append(p.get_text())

In [76]:
driver.back() ##되돌아오는 것

In [77]:

textlist =[]
for li in lis:
    sub_url = li.select_one('a._searchListTitleAnchor')['href']
    driver.get(sub_url)
    time.sleep(2)
    count = int(driver.find_element_by_css_selector('._answerCount.num').text)
    anwer_page = (count-1) // 5 #5개일때 문제생겨서 한개뻄
    for i in range(anwer_page):
        driver.find_element_by_id('nextPageButton').click()
        time.sleep(2)
    
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')

    divs = soup.select('.answer-content__item._contentWrap._answer')
    for div in divs:
        ps = div.select('.se-module.se-module-text p')

        for p in ps:
            text =p.get_text()
            if text== ' ' or text== '\u200b':
                continue
            textlist.append(text)

    driver.back()
    time.sleep(2)

In [79]:
len(textlist)

983

In [82]:
textlist =[]
for page in range(1,11):
    print(page)
    base_url ='https://kin.naver.com/search/list.naver?query='
    url = base_url + quote('여친선물') + f'&page={page}'
    driver.get(url)
    time.sleep(2)
    
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')

    lis = soup.select('.basic1 li')
    
    for li in lis:
        sub_url = li.select_one('a._searchListTitleAnchor')['href']
        driver.get(sub_url)
        time.sleep(2)
        count = int(driver.find_element_by_css_selector('._answerCount.num').text)
        anwer_page = (count-1) // 5 #5개일때 문제생겨서 한개뻄
        for i in range(anwer_page):
            driver.find_element_by_id('nextPageButton').click()
            time.sleep(2)
        
        html = driver.page_source
        soup = BeautifulSoup(html,'html.parser')

        divs = soup.select('.answer-content__item._contentWrap._answer')
        for div in divs:
            ps = div.select('.se-module.se-module-text p')

            for p in ps:
                text =p.get_text()
                if text== ' ' or text== '\u200b':
                    continue
                textlist.append(text)

        driver.back()
        time.sleep(2)


1
2
3
4
5
6
7
8
9
10


In [83]:
len(textlist)

4801

In [88]:
with open('data/new_gift.txt', 'w', encoding='utf-8') as fp:
    for text in textlist:
        fp.write(text+'\n',)

In [89]:
driver.close()